## <span style="color:#3182bd"><center>**Dividir, Extraer texto, Renombrar y Encriptar PDF**</center></span>

##  **Introducción**

Este código divide un archivo en el número de hojas que le indiquemos, extrae la información con los datos que le ordenemos para que encripte y renombre cada PDF dividido. Importante todos los archivos deben estar en formato PDF. 

${}$

In [1]:
# se deben tener instaladios los siguientes paquetes
# pip install os
# pip install openpyxl
# pip install pandas
# pip install PyMuPDF
# pip install PyPDF2

## Dividir PDF

El siguiente código divide un archivo en formato PDF en grupos de página que se le indiquen y les coloca nombres ascendentes comenzando del número 1.

1. Modificar la ruta **archivo_pdf_entrada** por la ruta del Pdf a dividir. 

2. Creamos una carpeta donde vamos a dejar los pdf ya divididos y modificamos la **ruta-salida**.

3. Modificamos el valor de **paginas_por_grupo** colocando cada cuantas hojas queremos que se divida el archivo.


In [38]:
import PyPDF2
import os

archivo_pdf_entrada = "ruta"
ruta_salida = "ruta"
paginas_por_grupo = 3


with open(archivo_pdf_entrada, "rb") as archivo:
    pdf = PyPDF2.PdfReader(archivo)
    num_paginas = len(pdf.pages)

    for grupo_numero in range(0, num_paginas, paginas_por_grupo):
        nuevo_pdf = PyPDF2.PdfWriter()

        for pagina_numero in range(grupo_numero, min(grupo_numero + paginas_por_grupo, num_paginas)):
            pagina = pdf.pages[pagina_numero]
            nuevo_pdf.add_page(pagina)

        nombre_archivo_salida = f"{grupo_numero // paginas_por_grupo + 1}.pdf" 
        ruta_completa_salida = os.path.join(ruta_salida, nombre_archivo_salida)

        with open(ruta_completa_salida, "wb") as nuevo_archivo:
            nuevo_pdf.write(nuevo_archivo)

print("Archivos divididos")


Archivos divididos


## Dividir varios pdf 

El siguiente código selecciona todos los archivos que se encuentran en una carpeta en formato PDF, extrae la página que le indiquemos y lo guarda en una ruta diferente.

1. Modificamos la ruta **directorio_entrada** por la dirección de la carpeta donde se encuentran los formularios.

2. Modificamos la ruta **directorio_salida** por la dirección de la carpeta donde queremos dejar los PDF extraidos.

3. Modificamos **pagina_a_extraer** con el número de la página que queremos extraer del PDF.


In [2]:
import PyPDF2
import os


directorio_entrada = "ruta"

directorio_salida = "ruta"

pagina_a_extraer = 1  


if not os.path.exists(directorio_salida):
    os.makedirs(directorio_salida)


for archivo_pdf_entrada in os.listdir(directorio_entrada):
    if archivo_pdf_entrada.endswith(".pdf"):
        
        ruta_pdf_entrada = os.path.join(directorio_entrada, archivo_pdf_entrada)

        with open(ruta_pdf_entrada, "rb") as archivo:
            pdf = PyPDF2.PdfReader(archivo)

            if pagina_a_extraer <= len(pdf.pages):
                nuevo_pdf = PyPDF2.PdfWriter()
                pagina = pdf.pages[pagina_a_extraer - 1]  
                nuevo_pdf.add_page(pagina)

                
                nombre_archivo_salida = f"{pagina_a_extraer}_{archivo_pdf_entrada}"
                ruta_completa_salida = os.path.join(directorio_salida, nombre_archivo_salida)

                with open(ruta_completa_salida, "wb") as nuevo_archivo:
                    nuevo_pdf.write(nuevo_archivo)

print("Archivos divididos y renombrados")


## Extraer Información del PDF

El siguiente código extrae la información que le indiquemos en las coordenadas, posteriormente realiza un proceso de limpieza a los datos, para dejar la información en un dataframe y  guardar en un archivo de Excel la información extraída.

1. En la **carpeta _pdf** colocar la ruta de los PDF a los cuales se le va a extraer la información. 

2. Crear un archivo de Excel y colocar la ruta en **nombre_archivo**

In [12]:
import fitz
import pandas as pd
import os
import re

carpeta_pdf = "ruta"
nombre_archivo = "ruta"
nombre_hoja = "Hoja1"

coordenadas = [
    {"nombre": "poliza", "coordenada": (45, 79, 72, 87)},
    {"nombre": "nombre", "coordenada": (98, 141, 227, 149)},       
    {"nombre": "id", "coordenada": (494, 128, 535, 137)}, 
    {"nombre": "empresa", "coordenada": (99, 100, 147, 108)},    
    {"nombre": "inicio_vigencia", "coordenada": (146, 181, 195, 188)},
    {"nombre": "fin_vigencia", "coordenada": (242, 181, 291, 188)},
    {"nombre": "valor_asegurado", "coordenada": (380, 282, 417, 289)},
    {"nombre": "nombre_Beneficiario_1", "coordenada": (98, 141, 227, 149)},
    {"nombre": "direccion", "coordenada": (45, 253, 137, 261)},
    {"nombre": "ciudad", "coordenada": (226, 253, 256, 261)},
    {"nombre": "departamento", "coordenada": (343, 253, 400, 264)},
    {"nombre": "pais", "coordenada": (485, 253, 522, 261)}
]

df_resultado = pd.DataFrame(columns=[item["nombre"] for item in coordenadas])

for archivo_pdf in os.listdir(carpeta_pdf):
    if archivo_pdf.endswith(".pdf"):
        
        pdf_file = os.path.join(carpeta_pdf, archivo_pdf)
        doc = fitz.open(pdf_file)
        page_num = 0  

        if page_num >= 0 and page_num < doc.page_count:
            page = doc[page_num]
            tex_coor = {}
            
            for i, item in enumerate(coordenadas):
                nombre = item["nombre"]
                x0, y0, x1, y1 = item["coordenada"]
                text = page.get_text("text", clip=(x0, y0, x1, y1))
                tex_coor[nombre] = [text]
            df = pd.DataFrame(tex_coor)
            df_resultado = pd.concat([df_resultado, df], ignore_index=True)
            doc.close()
        else:
            print(f"La página {page_num + 1} no es válida para el PDF: {pdf_file}")

patron = r'[\n.$]'
columnas_a_procesar = [
    'poliza', 'nombre', 'id', 'empresa', 'inicio_vigencia', 'fin_vigencia', 'valor_asegurado',
    'nombre_Beneficiario_1', "direccion","ciudad","departamento","pais"  
]
for columna in columnas_a_procesar:
    df_resultado[columna] = df_resultado[columna].apply(lambda x: re.sub(patron, '', x).rstrip().lstrip())
df_resultado.to_excel(nombre_archivo, sheet_name=nombre_hoja, index=False)

df_resultado.head()

,poliza,nombre,id,empresa,inicio_vigencia,fin_vigencia,valor_asegurado,nombre_Beneficiario_1,direccion,ciudad,departamento,pais
0,1003960,LOPEZ ROZO GEIMNIR YAZMIN,53133039,UNIVERSIDAD,01/MARZO/2023,01/MARZO/2024,"6,056,84000",LOPEZ ROZO GEIMNIR YAZMIN,CARRERA 69 BIS NO 3A 06,BOGOTA,BOGOTA,COLOMBIA
1,1003960,ACOSTA GONZALEZ LUIS ALEJANDRO,79797489,UNIVERSIDAD,01/MARZO/2023,01/MARZO/2024,"6,056,84000",ACOSTA GONZALEZ LUIS ALEJANDRO,CL 3 3E 116 AP 501 TO 10,CAJICA,CUNDINAMARCA,COLOMBIA


## Encriptar y renombrar pdf

1. En la **carpeta_origen** se debe colocar la ruta dónde se encuentran los archivos divididos.  
2. Crear la **carpeta_destino** y colocar la ruta. 
3. En **ruta_excel** colocar la ruta del archivo de Excel dónde se encuentran los nombres y las claves de los archivos a renombrar.  

In [1]:
import os
import openpyxl
import pandas as pd
import PyPDF2

carpeta_origen = "ruta"
carpeta_destino = "ruta"
ruta_excel = "ruta"

workbook = openpyxl.load_workbook(ruta_excel)
hoja_excel = workbook["Hoja1"]
df_id = pd.read_excel(ruta_excel)
contrasenas = [str(i) for i in df_id["id"]]
nombre = [str(i) for i in df_id["nombre"]]

archivos_pdf = [archivo for archivo in os.listdir(carpeta_origen) if archivo.endswith(".pdf")]

for i, archivo_pdf in enumerate(archivos_pdf):
    ruta_pdf_entrada = os.path.join(carpeta_origen, archivo_pdf)
    ruta_pdf_salida = os.path.join(carpeta_destino, f"{i+1}_{nombre[i]}.pdf")
  
    with open(ruta_pdf_entrada, "rb") as archivo_original:
        pdf = PyPDF2.PdfReader(archivo_original)
        pdf_writer = PyPDF2.PdfWriter()
        for pagina_num in range(len(pdf.pages)):
            pdf_writer.add_page(pdf.pages[pagina_num])
        pdf_writer.encrypt(contrasenas [i])
        
        with open(ruta_pdf_salida, "wb") as archivo_encriptado:
            pdf_writer.write(archivo_encriptado)
workbook.close()
print("Docuemntos encriptados con el ID del funcionario y renombrados")

Docuemntos encriptados con el ID del funcionario y renombrados


## Solo renombrar 

Esta función solo funciona para renombrar los archivos que se encuencuentran en formato PDF, no los encripta. Para esto utilizaremos el siguiente código:

1.  En **carpeta_origen** colocamos la ruta de la carpeta donde tenemos guardados los archivos PDF que deseamos renombrar.

2.  Creamos la carpeta dónde vamos a dejar los archivos PDF renombrados y modificamos la ruta **carpeta_destino**

3.  La ruta del Excel dónde tenemos los nombres de los archivos a renombrar la copiamos en **ruta_excel**


In [1]:
import os
import openpyxl
import pandas as pd

carpeta_origen = "ruta"
carpeta_destino = "ruta"
ruta_excel = "ruta"

workbook = openpyxl.load_workbook(ruta_excel)
hoja_excel = workbook["Hoja1"]

df_id = pd.read_excel(ruta_excel)
nombre = [str(i) for i in df_id["nombre"]]

archivos_pdf = [archivo for archivo in os.listdir(carpeta_origen) if archivo.endswith(".pdf")]

for i, archivo_pdf in enumerate(archivos_pdf):
    ruta_pdf_entrada = os.path.join(carpeta_origen, archivo_pdf)
    ruta_pdf_salida = os.path.join(carpeta_destino, f"{i+1}_{nombre[i]}.pdf")

    os.rename(ruta_pdf_entrada, ruta_pdf_salida)

workbook.close()

print("Documentos Renombrados")

InvalidFileException: openpyxl does not support  file format, please check you can open it with Excel first. Supported formats are: .xlsx,.xlsm,.xltx,.xltm